This output parser allows users to specify an arbitrary JSON schema and query LLMs for outputs that conform to that schema.

Keep in mind that large language models are leaky abstractions! You'll have to use an LLM with sufficient capacity to generate well-formed JSON. In the OpenAI family, DaVinci can do reliably but Curie's ability already drops off dramatically.

You can optionally use Pydantic to declare your data model.

In [ ]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI

In [ ]:
model = ChatOpenAI(model= "gpt-4o-mini",temperature=0)

In [ ]:
# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

In [ ]:
class Response(BaseModel):
    ai_reasoning: str = Field(description="AI reasoning for the response")
    tldr: str = Field(description="TLDR of the ai reasoning")

parser = JsonOutputParser(pydantic_object=Response)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

print(prompt.partial_variables['format_instructions'])

In [ ]:
# And a query intented to prompt a language model to populate the data structure.
joke_query = "Tell me a recipe of beef soup."

# Set up a parser + inject instructions into the prompt template.
parser = JsonOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser

chain.invoke({"query": joke_query})

## Streaming

In [ ]:
for s in chain.stream({"query": joke_query}):
    print(s)

## Without Pydantic
You can also use this without Pydantic. This will prompt it return JSON, but doesn't provide specific about what the schema should be.

In [ ]:
joke_query = "Tell me a joke."

parser = JsonOutputParser()

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser

chain.invoke({"query": joke_query})